<a href="https://colab.research.google.com/github/aryixa/closetai/blob/main/closetai1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch torchvision scikit-learn opencv-python pillow matplotlib


In [3]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import cv2
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from google.colab import files


In [ ]:
from google.colab import files
uploaded = files.upload()

In [4]:
model = models.resnet18(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])
model.eval()

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 68.2MB/s]


In [5]:
def get_embedding(img_path):
    img = Image.open(img_path).convert("RGB")
    img = transform(img).unsqueeze(0)
    with torch.no_grad():
        emb = model(img)
    return emb.flatten().numpy()


In [6]:
def dominant_color(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (50,50))
    pixels = img.reshape((-1,3))
    avg = np.mean(pixels, axis=0)
    return avg


In [7]:
import os

image_paths = [f for f in os.listdir() if f.endswith(('jpg','png','jpeg'))]
embeddings = []

for p in image_paths:
    embeddings.append(get_embedding(p))

print("Dataset size:", len(embeddings))


Dataset size: 0


In [ ]:
query_file = files.upload()
query_name = list(query_file.keys())[0]


In [ ]:
query_emb = get_embedding(query_name)
sims = cosine_similarity([query_emb], embeddings)[0]
top3_idx = np.argsort(sims)[-3:]

print("Top Similar Images:")
for idx in top3_idx:
    print(image_paths[idx], "Similarity:", sims[idx])


In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(1,4,1)
plt.imshow(Image.open(query_name))
plt.title("Query")
plt.axis('off')

for i, idx in enumerate(top3_idx):
    plt.subplot(1,4,i+2)
    plt.imshow(Image.open(image_paths[idx]))
    plt.title(f"Match {i+1}")
    plt.axis('off')

plt.show()


In [ ]:
color = dominant_color(query_name)
print("Dominant RGB:", color)


In [ ]:
!pip -q install faiss-cpu sentence-transformers transformers


In [ ]:
fashion_docs = [
    "For summer weddings, choose breathable fabrics like cotton, linen, chiffon. Prefer pastel shades.",
    "For formal events, avoid loud prints; choose solid colors, clean silhouettes, minimal accessories.",
    "For a casual brunch, light colors and relaxed fits work well. Sneakers or flats are ideal.",
    "For office wear, stick to neutral colors, modest fits, and structured pieces like blazers or trousers.",
    "If the outfit has a heavy pattern, balance with solid pieces. If it's solid, add a patterned accessory.",
    "Warm undertones pair well with earthy shades (beige, olive, mustard). Cool undertones suit blues, greys, lavender.",
    "Monochrome outfits look elegant. Use one accent item like a bag or shoes for contrast.",
    "Denim is casual; avoid denim for formal events unless styled with a blazer and dressy footwear."
]


In [ ]:
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

text_embedder = SentenceTransformer("all-MiniLM-L6-v2")

doc_embs = text_embedder.encode(fashion_docs, convert_to_numpy=True, normalize_embeddings=True)

dim = doc_embs.shape[1]
index = faiss.IndexFlatIP(dim)  # Inner product works well after normalization (same as cosine)
index.add(doc_embs)

print("Vector DB ready. Docs indexed:", index.ntotal)


In [ ]:
def retrieve_fashion_context(user_query, k=3):
    q_emb = text_embedder.encode([user_query], convert_to_numpy=True, normalize_embeddings=True)
    scores, idx = index.search(q_emb, k)
    retrieved = [fashion_docs[i] for i in idx[0]]
    return retrieved, scores[0]


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-base"

tok = AutoTokenizer.from_pretrained(model_name)
t5 = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
t5 = t5.to(device)

print("Loaded:", model_name, "| device:", device)


In [ ]:
def generate_t5(prompt, max_new_tokens=220, min_new_tokens=90):
    inputs = tok(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        out = t5.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            min_new_tokens=min_new_tokens,
            num_beams=6,
            repetition_penalty=1.7,
            no_repeat_ngram_size=4,
            length_penalty=1.0,
            early_stopping=True
        )
    return tok.decode(out[0], skip_special_tokens=True)


In [ ]:
def rag_stylist_answer(user_query, k=3):
    retrieved, scores = retrieve_fashion_context(user_query, k=k)
    rules = "\n".join([f"Rule {i+1}: {r}" for i, r in enumerate(retrieved)])

    prompt = f"""
Use ONLY these rules. Write ONE clear answer (no repeating words).

{rules}

Question: {user_query}

Fill this exactly with natural sentences:

Outfit: <one outfit suggestion in 2 sentences>
Reason: <1-2 sentences citing Rule numbers like Rule 1>
Tips: <3 bullet points, each short>
"""
    out = generate_t5(prompt, max_new_tokens=220, min_new_tokens=90)
    return out, retrieved, scores


In [ ]:
query = "What should I wear to a summer wedding as a college student?"
answer, retrieved, scores, prompt = rag_stylist_answer_debug(query, k=3)

print("PROMPT USED:\n", prompt)
print("\nAI ANSWER:\n", answer)
